In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import gc

print_to_file = False 
test_run = False 



# train = pd.read_pickle(ENV.lightgbm_train_764.value)
train = pd.read_pickle(ENV.lgb_train_0827.value)
print('train shape is: {}'.format(train.shape))
# test = pd.read_pickle(ENV.lightgbm_test_764.value)
test = pd.read_pickle(ENV.lgb_test_0827.value)
print('test shape is: {}'.format(test.shape))
fe_id = 'comb_764'

train shape is: (307511, 409)
test shape is: (48744, 408)


In [2]:
train['SK_ID_CURR'] = train['SK_ID_CURR'].astype(int)
test['SK_ID_CURR'] = test['SK_ID_CURR'].astype(int)
targets = train.TARGET.values

train_id = train['SK_ID_CURR']
test_id = test['SK_ID_CURR']

X = pd.concat([train.drop('TARGET',axis=1),test])

In [3]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


In [4]:
def extreme_fill_na(X,th=30,bucket=63,convert_category=True):
    for col in tqdm(X.columns):
        X[col] = X[col].replace(np.NINF,np.NAN)
        X[col] = X[col].replace(np.PINF,np.NAN)
        X[col] = X[col].replace(np.Inf,np.NAN)
        convertBack = False
        
        if X[col].nunique() < th and convert_category:
            X[col] = X[col].astype('category')
        
        if X[col].isnull().sum() == 0:
            continue
        else:
            if str(X[col].dtypes) == 'category':
                convertBack = True
                X[col] = X[col].astype('float')
            fill_na =X[col].min() - (X[col].max() - X[col].min()) / bucket
            X[col] = X[col].fillna(fill_na)
            if convertBack:
                X[col] = X[col].astype('category')
    return X
            
        



def convert_category(X,th=40):
    for col in tqdm(X.columns):
        if X[col].nunique()<th:
            X[col] = X[col].astype('category')
    return X
            
def saving(X):
    print('saving...')
    train_save = X.iloc[:307511].copy()
    train_save['TARGET'] = targets
    print(train_save.shape)

    test_save = X.iloc[307511:].copy()
    train_save.to_pickle(ENV.lgb_train_0827_na_extrem.value)
    test_save.to_pickle(ENV.lgb_test_0827_na_extrem.value)

In [6]:
X = extreme_fill_na(X)

100%|██████████| 408/408 [00:07<00:00, 53.00it/s]


In [8]:
saving(X)

saving...
(307511, 409)
